# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
L. Eisert  ->  L. Eisert  |  ['L. Eisert']
J. Mah  ->  J. Mah  |  ['J. Mah']
Arxiv has 56 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2308.14793


extracting tarball to tmp_2308.14793...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

Found 322 bibliographic references in tmp_2308.14793/ms.bbl.
syntax error in line 501: '=' expected
Retrieving document from  https://arxiv.org/e-print/2308.15128


extracting tarball to tmp_2308.15128...

 done.


J. Mah  ->  J. Mah  |  ['J. Mah']


Found 72 bibliographic references in tmp_2308.15128/main.bbl.
syntax error in line 310: '}' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.14793-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.14793) | **IllustrisTNG in the HSC-SSP: image data release and the major role of  mini mergers as drivers of asymmetry and star formation**  |
|| C. Bottrell, et al. -- incl., <mark>A. Pillepich</mark>, <mark>L. Eisert</mark> |
|*Appeared on*| *2023-08-30*|
|*Comments*| *32 pages; 18 figures; submitted to MNRAS; Image data available via the TNG website: www.tng-project.org/bottrell23*|
|**Abstract**| At fixed galaxy stellar mass, there is a clear observational connection between structural asymmetry and offset from the star forming main sequence, $\Delta$SFMS. Herein, we use the TNG50 simulation to investigate the relative roles of major mergers (stellar mass ratios $\mu\geq0.25$), minor ($0.1 \leq \mu < 0.25$), and mini mergers ($0.01 \leq \mu < 0.1$) in driving this connection amongst star forming galaxies (SFGs). We use dust radiative transfer post-processing with SKIRT to make a large, public collection of synthetic Hyper Suprime-Cam Subaru Strategic Program (HSC-SSP) images of simulated TNG galaxies over $0.1\leq z \leq 0.7$ with $\log M_{\star} / \mathrm{M}_{\odot}\geq9$ ($\sim750$k images). Using their instantaneous SFRs, known merger histories/forecasts, and HSC-SSP asymmetries, we show (1) that TNG50 SFGs qualitatively reproduce the observed trend between $\Delta$SFMS and asymmetry and (2) a strikingly similar trend emerges between $\Delta$SFMS and the time-to-coalescence for mini mergers. Controlling for redshift, stellar mass, environment, and gas fraction, we show that individual mini merger events yield small enhancements in SFRs and asymmetries that are sustained on long timescales (at least $\sim3$ Gyr after coalescence, on average) -- in contrast to major/minor merger remnants which peak at much greater amplitudes but are consistent with controls only $\sim1$ Gyr after coalescence. Integrating the boosts in SFRs and asymmetries driven by $\mu\geq0.01$ mergers since $z=0.7$ in TNG50 SFGs, we show that mini mergers are responsible for (i) $55$ per cent of all merger-driven star formation and (ii) $70$ per cent of merger-driven asymmetric structure. Due to their relative frequency and prolonged boost timescales, mini mergers dominate over their minor and major counterparts in driving star formation and asymmetry in SFGs. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.15128-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.15128) | **Close-in ice lines and the super-stellar C/O ratio in discs around very  low-mass stars**  |
|| <mark>J. Mah</mark>, B. Bitsch, I. Pascucci, T. Henning |
|*Appeared on*| *2023-08-30*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| The origin of the elevated C/O ratios in discs around late M dwarfs compared to discs around solar-type stars is not well understood. Here we endeavour to reproduce the observed differences in the disc C/O ratios as a function of stellar mass using a viscosity-driven disc evolution model and study the corresponding atmospheric composition of planets that grow inside the water-ice line in these discs. We carried out simulations using a coupled disc evolution and planet formation code that includes pebble drift and evaporation. We used a chemical partitioning model for the dust composition in the disc midplane. Inside the water-ice line, the disc's C/O ratio initially decreases to sub-stellar due to the inward drift and evaporation of water-ice-rich pebbles before increasing again to super-stellar values due to the inward diffusion of carbon-rich vapour. We show that this process is more efficient for very low-mass stars compared to solar-type stars due to the closer-in ice lines and shorter disc viscous timescales. In high-viscosity discs, the transition from sub-stellar to super-stellar takes place faster due to the fast inward advection of carbon-rich gas. Our results suggest that planets accreting their atmospheres early (when the disc C/O is still sub-stellar) will have low atmospheric C/O ratios, while planets that accrete their atmospheres late (when the disc C/O has become super-stellar) can obtain high C/O ratios. Our model predictions are consistent with observations, under the assumption that all stars have the same metallicity and chemical composition, and that the vertical mixing timescales in the inner disc are much shorter than the radial advection timescales. This further strengthens the case for considering stellar abundances alongside disc evolution in future studies that aim to link planet (atmospheric) composition to disc composition. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2308.14793.md
    + _build/html/tmp_2308.14793/./Figures/IllustrisTNG50_1_MergersInf_RA_Isolated_3Gyr.png
    + _build/html/tmp_2308.14793/./Figures/IllustrisTNG50_1_MergersInf_SFR_Isolated_3Gyr.png
    + _build/html/tmp_2308.14793/./Figures/TypeFraction_RA_NoDoubles.png
    + _build/html/tmp_2308.14793/./Figures/SSFR_Mstar_091_ResidualAsymmetry_HighRes.jpg
exported in  _build/html/2308.15128.md
    + _build/html/tmp_2308.15128/./discCO_C60_5ms.png
    + _build/html/tmp_2308.15128/./disc_tvisc.png
    + _build/html/tmp_2308.15128/./disc_temp.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gimtwod}{\textsc{gim2d}}$
$\newcommand{\sersic}{sérsic}$
$\newcommand{\Sersic}{Sérsic}$
$\newcommand{\sextractor}{\textsc{SExtractor}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\}{nuprocess}$
$\newcommand{\}{nodata}$
$\newcommand{ ◦ee}{^{\circ}}$
$\newcommand{\}{Msolar}$
$\newcommand{\}{alphafe}$
$\newcommand{\}{na}$
$\newcommand{\}{HI}$
$\newcommand{\}{sion}$
$\newcommand{\}{vninety}$
$\newcommand{\}{Lbol}$
$\newcommand{\}{Mstar}$
$\newcommand{\}{logMstar}$
$\newcommand{\}{logRimp}$
$\newcommand{\}{logLbol}$
$\newcommand{\}{logsSFR}$
$\newcommand{\}{kms}$
$\newcommand{\}{zabs}$
$\newcommand{\}{zem}$
$\newcommand{\}{Rimp}$
$\newcommand{\}{Rvir}$
$\newcommand{\}{deltaEW}$
$\newcommand{\}{RadRat}$
$\newcommand{\}{mnfe}$
$\newcommand{\}{Mgeqw}$
$\newcommand{\}{Feeqw}$
$\newcommand{\}{fracMgFe}$
$\newcommand{\}{omegaDLA}$
$\newcommand{\}{ndla}$
$\newcommand{\}{npdla}$
$\newcommand{\}{nlpdla}$
$\newcommand{\}{nxpdla}$
$\newcommand{\}{nmdla}$
$\newcommand{\}{nlmdla}$
$\newcommand{\}{nxmdla}$
$\newcommand{\}{CosmoZ}$
$\newcommand{\}{fNX}$
$\newcommand{\}{rifs}$
$\newcommand{\}{realsim}$
$\newcommand{\}{tpost}$
$\newcommand{\}{dtc}$
$\newcommand{\}{dtcoal}$
$\newcommand{\}{skirt}$
$\newcommand{\}{deltasfms}$
$\newcommand{\}{dlogsfms}$
$\newcommand{\}{fgas}$
$\newcommand{\}{mstar}$
$\newcommand{\}{logsfr}$
$\newcommand{\}{dlogsfr}$
$\newcommand{\}{logssfr}$
$\newcommand{\}{dra}$
$\newcommand{\}{dsfr}$
$\newcommand{\}{dtcoffset}$
$\newcommand{\}{dtcoaloffset}$
$\newcommand{\}{dsfms}$</div>



<div id="title">

# IllustrisTNG in the HSC-SSP: image data release and the major role of mini mergers as drivers of asymmetry and star formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.14793-b31b1b.svg)](https://arxiv.org/abs/2308.14793)<mark>Appeared on: 2023-08-30</mark> -  _32 pages; 18 figures; submitted to MNRAS; Image data available via the TNG website: www.tng-project.org/bottrell23_

</div>
<div id="authors">

C. Bottrell, et al. -- incl., <mark>A. Pillepich</mark>, <mark>L. Eisert</mark>

</div>
<div id="abstract">

**Abstract:** At fixed galaxy stellar mass, there is a clear observational connection between structural asymmetry and offset from the star forming main sequence, $\dsfms$ . Herein, we use the TNG50 simulation to investigate the relative roles of major mergers (stellar mass ratios $\mu\geq0.25$ ), minor ( $0.1 \leq \mu < 0.25$ ), and mini mergers ( $0.01 \leq \mu < 0.1$ ) in driving this connection amongst star forming galaxies (SFGs). We use dust radiative transfer post-processing with SKIRT to make a large, public collection of synthetic Hyper Suprime-Cam Subaru Strategic Program (HSC-SSP) images of simulated TNG galaxies over $0.1\leq z \leq 0.7$ with $\logMstar\geq9$ ( $\sim750$ k images). Using their instantaneous SFRs, known merger histories/forecasts, and HSC-SSP asymmetries, we show (1) that TNG50 SFGs qualitatively reproduce the observed trend between $\dsfms$ and asymmetry and (2) a strikingly similar trend emerges between $\dsfms$ and the time-to-coalescence for mini mergers. Controlling for redshift, stellar mass, environment, and gas fraction, we show that individual mini merger events yield small enhancements in SFRs and asymmetries that are sustained on long timescales (at least $\sim3$ Gyr after coalescence, on average) -- in contrast to major/minor merger remnants which peak at much greater amplitudes but are consistent with controls only $\sim1$ Gyr after coalescence. Integrating the boosts in SFRs and asymmetries driven by $\mu\geq0.01$ mergers since $z=0.7$ in TNG50 SFGs, we show that mini mergers are responsible for (i) $55$ per cent of all merger-driven star formation and (ii) $70$ per cent of merger-driven asymmetric structure. Due to their relative frequency and prolonged boost timescales, mini mergers dominate over their minor and major counterparts in driving star formation and asymmetry in SFGs.

</div>

<div id="div_fig1">

<img src="tmp_2308.14793/./Figures/IllustrisTNG50_1_MergersInf_RA_Isolated_3Gyr.png" alt="Fig16.1" width="50%"/><img src="tmp_2308.14793/./Figures/IllustrisTNG50_1_MergersInf_SFR_Isolated_3Gyr.png" alt="Fig16.2" width="50%"/>

**Figure 16. -** $\dra$ vs $\dtcoal$ in individual mergersResidual asymmetry enhancements, $\dra$, and SFR enhancements, $\dlogsfr$, in individual mergers compared to non-merging controls as a function of merger $\dtcoal$ in TNG50. Galaxies are considered individual mergers if they have *exactly* one coalescence event within $|\dtcoal|<3$ Gyr and no other mergers from other mass ratio range. Each individual merger is matched to a control which has no coalescence event satisfying $|\dtcoal|<3$ Gyr (see Section \ref{sec:iso_selection}). Mini mergers exhibit statistically enhanced asymmetries and SFRs across the full range in $\dtcoal$. The peak enhancement amplitude in asymmetry and SFR increases with mass ratio. However, the enhancements in minor and major mergers are short-lived after coalescence. The asymmetries of individual minor and major merger remnants are consistent with their controls for $\dtcoal>1$ Gyr. In contrast, mini mergers yield long-lasting, modest asymmetry enhancements. Similarly, the longevity of SFR enhancement declines with increasing mass ratio. The SFRs of individual minor and major merger remnants are consistent with their controls for  $\dtcoal>2$ Gyr and $\dtcoal>1$ Gyr, respectively. *Individual* mini mergers in TNG50 drive statistically significant changes to galaxy morphologies and SFRs. Mini mergers are not piggybacking on the enhancements driven by minor and major mergers. (*fig:iso_merg*)

</div>
<div id="div_fig2">

<img src="tmp_2308.14793/./Figures/TypeFraction_RA_NoDoubles.png" alt="Fig3" width="100%"/>

**Figure 3. -** $R_A$ by merger typeStatistics of merging SFGs near coalescence, $| \dtcoal |<1.5$ Gyr, as a function of HSC residual asymmetry for $z\leq 0.3$. *Upper panel*: The merger fraction in each bin, $f_\mu$, split into major, minor, and mini regimes. The unshaded area shows the fraction of SFGs that have no $\mu\geq0.01$ merger within $1.5$ Gyr. Compared to minor and major mergers combined, mini mergers dominate for $R_A<0.3$. $68$ per cent of galaxies at $R_A>0.3$ are close to a merger -- of which $47$ per cent are mini mergers with no major or minor merger within $1.5$ Gyr. $32$ per cent of galaxies with asymmetries $R_A>0.3$ are non-mergers. *Middle panel*: Merger type fractions relative to $f_\mu(R_A\approx0)$(grey band). The fraction of high-$\mu$ mergers increases more rapidly with $R_A$ than low-$\mu$ mergers. *Lower panel*: Completeness and purity curves as a function of asymmetry threshold, $R_A>X$. Dark lines show statistics for all $\mu>0.01$ mergers. (*fig:ra_mergers*)

</div>
<div id="div_fig3">

<img src="tmp_2308.14793/./Figures/SSFR_Mstar_091_ResidualAsymmetry_HighRes.jpg" alt="Fig11" width="100%"/>

**Figure 11. -** SFMS and Residual AsymmetryStar-forming main sequence (SFMS) fitting, star forming galaxy (SFG) selection, and the relationship between SFMS offset and galaxy residual asymmetries for TNG50 galaxies at $z=0.1$. The left panel shows the full sample of 12,088 galaxies ($3,047$ galaxies $\times 4$ sightlines). The solid black line shows the linear fit to the SFMS described in Section \ref{sec:sfms} and in Table \ref{tab:sfms}. The dotted line shows $\dsfms = \log(\mathrm{SSFR}) - \langle \log(\mathrm{SSFR})\rangle_{\mathrm{MS}} = -1$ dex. We define SFGs as those above this line. Each galaxy is coloured by its residual asymmetry, $R_A$(measured after removal of the best-fitting Sérsic model). For visual purposes, an artificial $0.05$ dex scatter in SFR and $M_{\star}$ is introduced such that different orientations of the same intrinsic galaxy are not directly atop each other. The right panel shows 2D locally weighted regression in $R_A$ for the star-forming sample (LOESS: \citealt{1988Cleveland2DLoess,2013MNRAS.432.1862C}). The trend between $\dsfms$ and $R_A$ is qualitatively similar to that shown by \cite{2021ApJ...923..205Y}. (*fig:sfms_ra*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.14793"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Close-in ice lines and the super-stellar C/O ratio in discs around very low-mass stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.15128-b31b1b.svg)](https://arxiv.org/abs/2308.15128)<mark>Appeared on: 2023-08-30</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

<mark>J. Mah</mark>, B. Bitsch, I. Pascucci, T. Henning

</div>
<div id="abstract">

**Abstract:** The origin of the elevated C/O ratios in discs around late M dwarfs compared to discs around solar-type stars is not well understood. Here we endeavour to reproduce the observed differences in the disc C/O ratios as a function of stellar mass using a viscosity-driven disc evolution model and study the corresponding atmospheric composition of planets that grow inside the water-ice line in these discs. We carried out simulations using a coupled disc evolution and planet formation code that includes pebble drift and evaporation. We used a chemical partitioning model for the dust composition in the disc midplane. Inside the water-ice line, the disc's C/O ratio initially decreases to sub-stellar due to the inward drift and evaporation of water-ice-rich pebbles before increasing again to super-stellar values due to the inward diffusion of carbon-rich vapour. We show that this process is more efficient for very low-mass stars compared to solar-type stars due to the closer-in ice lines and shorter disc viscous timescales. In high-viscosity discs, the transition from sub-stellar to super-stellar takes place faster due to the fast inward advection of carbon-rich gas. Our results suggest that planets accreting their atmospheres early (when the disc C/O is still sub-stellar) will have low atmospheric C/O ratios, while planets that accrete their atmospheres late (when the disc C/O has become super-stellar) can obtain high C/O ratios. Our model predictions are consistent with observations, under the assumption that all stars have the same metallicity and chemical composition, and that the vertical mixing timescales in the inner disc are much shorter than the radial advection timescales. This further strengthens the case for considering stellar abundances alongside disc evolution in future studies that aim to link planet (atmospheric) composition to disc composition.

</div>

<div id="div_fig1">

<img src="tmp_2308.15128/./discCO_C60_5ms.png" alt="Fig1" width="100%"/>

**Figure 1. -** Temporal evolution of the gas disc C/O ratio (normalised to the stellar value) as a function of distance from the star, stellar mass (left to right), and disc viscosity (top and bottom rows). Ice lines are marked by vertical dashed white lines, and solid black lines indicate an absolute C/O ratio of 1. The carbon line is absent in the top leftmost panel because it is located inside 0.1 AU. Outside the CO ice line ($T_{\rm disc} \leq 20 {\rm K}$; white region in the subplots), all chemical species are assumed to be frozen in the solid phase. (*fig:discCO_C60_5ms*)

</div>
<div id="div_fig2">

<img src="tmp_2308.15128/./disc_tvisc.png" alt="Fig5" width="100%"/>

**Figure 5. -** Comparison of disc properties for a disc around an M dwarf and a solar-type star in our model. _ Left_: Aspect ratio of a disc around a $0.1 M_{\odot}$ star (black lines) and a disc around a $1.0 M_{\odot}$ star (orange lines) for low (solid lines) and high (dashed lines) disc viscosities. _ Right_: Viscous accretion timescale of the discs as a function of radial distance, stellar mass, and disc viscosity. Solid circles indicate the location of the water-ice (blue) and $CH_4$-ice (red) lines in each disc. (*fig:disc_tvisc*)

</div>
<div id="div_fig3">

<img src="tmp_2308.15128/./disc_temp.png" alt="Fig3" width="100%"/>

**Figure 3. -** Disc midplane temperature profile as a function of stellar mass and disc viscosity. The condensation temperatures of carbon- and oxygen-bearing species in the disc are marked with dashed grey lines. (*fig:disc_temp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.15128"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

257  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
